<a href="https://colab.research.google.com/github/talisma-cassoma/ourmoney_desktop/blob/main/Exploratory_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Perguntas e Objetivos**  

1. Como posso gastar melhor meu dinheiro?  
2. Com o que gasto mais e como estabelecer um budget fixo?  
3. Que gastos devo eliminar?  
4. Como criar objetivos financeiros com datas?  
5. Outras Perguntas e Objetivos Relevantes  


#### 1. **Como posso gastar melhor meu dinheiro?**  
- **Análise proposta**:  
  - **Classificação de despesas por categorias**: Identificar categorias de gastos recorrentes e calcular a proporção de cada categoria em relação à receita total.  
  - **Análise de correlação**: Avaliar se há relação entre categorias de despesas e períodos de maior ou menor receita.  
  - **Recomendação**: Usar a técnica de **"agrupamento" (clustering)** para descobrir combinações de gastos típicos (e.g., alimentação e transporte) e analisar sua sustentabilidade dentro da receita.  

In [26]:
import pandas as pd
import plotly.express as px

In [27]:
df_Transactions=pd.read_excel("/content/transactions.xlsx",sheet_name="Transactions").copy()
#delete unnecessary columns
df_Transactions = df_Transactions.drop(columns={'id','owner', 'email','synced'})
#delete space in begning and end and lowercase
df_Transactions['category'] = df_Transactions['category'].apply(lambda x: str(x).strip().lower())
df_Transactions['description'] = df_Transactions['description'].apply(lambda x: str(x).strip().lower())

tabela das transacoes mensal:

In [28]:
pd.options.display.float_format='{:.2f}'.format

In [35]:
 # Verifica se a coluna 'createdAt' está no formato de string
df_Transactions['createdAt'] = pd.to_datetime(df_Transactions['createdAt'], format='%d-%m-%Y', errors='coerce')
# Cria a nova coluna 'month' com o número do mês
df_Transactions['month'] = df_Transactions['createdAt'].dt.month

# Despesas por mês e categoria
expenses_by_month = df_Transactions[df_Transactions['type'] == 'outcome'].groupby(['month', 'category'])['price'].sum().reset_index()

# Receita por mês
df_total_by_month = df_Transactions[df_Transactions['type'] == 'outcome'].groupby('month')['price'].sum().reset_index(name='Total month expense')

df_total_by_month['perc %'] = (df_total_by_month['Total month expense'] / expenses_by_month['price']) * 100
# Unir as informações
expenses_by_month = expenses_by_month.merge(df_total_by_month, on='month', how='left')

# Calcular a porcentagem de cada categoria dentro do total mensal
expenses_by_month['perc %'] = (expenses_by_month['price'] / expenses_by_month['Total month expense']) * 100

# Map numeric months to alphabetic format
month_map = {
        1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June',
        7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
    }
expenses_by_month['month'] = expenses_by_month['month'].map(month_map)
expenses_by_month


,month,category,price,Total month expense,perc %
0,February,agua e luz,231.00,5583.70,4.14
1,February,alimentaçao,92.50,5583.70,1.66
2,February,aparelhos,20.00,5583.70,0.36
3,February,creche,1800.00,5583.70,32.24
4,February,divida,1950.00,5583.70,34.92
...,...,...,...,...,...
188,December,presentes,140.00,7953.99,1.76
189,December,roupas,1325.00,7953.99,16.66
190,December,talisma,804.00,7953.99,10.11
191,December,transporte,1543.00,7953.99,19.40


In [45]:
import pandas as pd

def create_pivot_table(dataframe):
    # Create DataFrame
    df = pd.DataFrame(dataframe)

    # Create pivot table
    pivot_table = pd.pivot_table(
        df,
        index=['category'],
        columns='month',
        fill_value=0
    )

    # Style the pivot table
    styled_table = (
        pivot_table.style
        .format("{:.2f}")  # Format all numeric values to two decimal places
        .background_gradient(cmap="coolwarm", axis=0)  # Add gradient coloring
        .set_caption("Annual Expenses by Category and Month")  # Add a title
        .set_table_styles([
            {'selector': 'caption', 'props': 'caption-side: top; font-size: 16px; font-weight: bold;'},
            {'selector': 'thead th', 'props': 'background-color: #f4f4f4; color: black; text-align: center;'},
            {'selector': 'th', 'props': 'font-size: 12px; text-align: center;'},
            {'selector': 'tbody th', 'props': 'border-bottom: 2px solid #000;'},  # Add border at the bottom of the header
            {'selector': 'td, th', 'props': 'border-right: 1px solid #ddd;'},  # Add borders between columns
            {'selector': 'th', 'props': 'border-left: 1px solid #ddd;'},  # Add left borders for the header
            {'selector': 'td', 'props': 'text-align: center; font-size: 11px;'},
        ])
    )

    # Return the styled table for display (works in Jupyter or other frontends)
    return styled_table

create_pivot_table(expenses_by_month)


tabela das transacoes anual

In [ ]:
df_total= df_Transactions.groupby('type')['price'].sum().reset_index()
df_total

,type,price
0,income,170121.0
1,outcome,168671.8


In [ ]:
# Filtrar apenas as despesas
df_expenses_categories = df_Transactions[df_Transactions['type'] == 'outcome']
# Filtrar apenas as receitas
df_incomes_categories = df_Transactions[df_Transactions['type'] == 'income']

# Calcular o total de despesas
total_expenses = df_expenses_categories['price'].sum()
# print results
print(f'# total of expenses: {total_expenses}')

# total of incomes
total_incomes =  df_incomes_categories['price'].sum()
print(f'total of incomes: {total_incomes}')

#total of each category
df_expenses_categories = df_expenses_categories.groupby('category')['price'].sum().reset_index()

# Adicionar a coluna de porcentagem
df_expenses_categories['perc % on expenses'] = (df_expenses_categories['price'] / total_expenses) * 100
# Adicionar a coluna de porcentagem
df_expenses_categories['perc % on income'] = (df_expenses_categories['price'] / total_incomes) * 100

# Visualizar o resultado
df_expenses_categories.head()


# total of expenses: 168671.8
total of incomes: 170121.0


,category,price,perc % on expenses,perc % on income
0,agua e luz,1974.67,1.170717,1.160744
1,ajuda,5.00,0.002964,0.002939
2,alimentaçao,19492.32,11.556360,11.457915
3,aluguel,30200.00,17.904593,17.752071
4,angola,8979.00,5.323356,5.278008


In [ ]:
#total of each category
df_incomes_categories = df_incomes_categories.groupby('category')['price'].sum().reset_index()

# Adicionar a coluna de porcentagem
df_incomes_categories['perc % on income'] = (df_incomes_categories['price'] / total_incomes) * 100

# Visualizar o resultado
df_incomes_categories.head()

,category,price,perc % on income
0,bolsa amci,2500.0,1.469542
1,divida,100.0,0.058782
2,emprestimo,7690.0,4.520312
3,extra,900.0,0.529035
4,geovana,50.0,0.029391


#### 2. **Com o que gasto mais e como estabelecer um budget fixo?**  
- **Análise proposta**:  
  - Identificar as 3 principais categorias de gastos e analise a média, mediana e desvio-padrão de cada uma para entender sua variação mensal.  
  - Utilizar uma abordagem de **envelope financeiro**, dividindo sua receita proporcionalmente para atender às necessidades, desejos e poupança.  
  - Criar simulações baseadas em limites orçamentários para prever impactos financeiros.  

#### 3. **Que gastos devo eliminar?**  
- **Critérios baseados em estatísticas**:  
  - Classificar as despesas com base no índice de prioridade, considerando frequência de uso, custo-benefício e relevância (e.g., compras por impulso).  
  - Utilizar uma análise de Pareto para identificar os 20% de gastos que consomem 80% do seu orçamento.  

#### 4. **Como criar objetivos financeiros com datas?**  
- **Análise proposta**:  
  - Definir metas de poupança ou investimento usando uma abordagem incremental baseada na média histórica de sua receita líquida.  
  - Utilizar **regressão linear** para projetar tendências futuras, ajustando os objetivos com base em padrões históricos de receita e despesa.  
  - Estabeleça **KPIs pessoais**:  
    - % de despesas essenciais vs. supérfluas.  
    - % de cumprimento de metas semanais/mensais.  
    - Taxa de poupança em relação à receita.  

---

### **Outras Perguntas e Objetivos Relevantes**  

1. **Qual o impacto sazonal nas minhas receitas e despesas?**  
   - Usar análises temporais para entender como as finanças variam por mês ou estação.  

2. **Quais são minhas despesas fixas e variáveis?**  
   - Classificararas despesas entre fixas (aluguel, contas) e variáveis (lazer, alimentação fora) para entender qual delas pode ser reduzida.  

3. **Qual a minha taxa de poupança?**  
   - Compare o total economizado com sua receita total ao longo do tempo e avalie a consistência.  

4. **Onde posso alocar melhor meus excedentes financeiros?**  
   - Sugestão: Calcular a relação entre investimentos realizados e retornos financeiros.  

5. **Quais categorias apresentam maior variabilidade de mês para mês?**  
   - Calcular a variância e o coeficiente de variação para identificar as categorias menos previsíveis.  

---
### **Próximos Passos**  

- Organizar suas transações financeiras por categoria, data e tipo (fixo/variável).  
- Definir métricas claras (e.g., % de redução em categorias específicas).  
- Implementar um método de acompanhamento contínuo para ajustar o planejamento conforme necessário.  